## Import modules

In [31]:
!pip install mxnet-mkl==1.6.0 numpy==1.23.1

In [32]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-jcgwm8ha/kobert-tokenizer_2c841a3f3f7c47c3a9653dfefba1186c
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-jcgwm8ha/kobert-tokenizer_2c841a3f3f7c47c3a9653dfefba1186c
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [33]:
!pip install gluonnlp

**Requirement**

* Python >= 3.6
* PyTorch >= 1.8.1
* transformers >= 4.8.2
* sentencepiece >= 0.1.91

In [34]:
#혹시 또 nlp에서 boolean에 대해 error가 생기면 처리할 코드
#np.bool = np.bool_

In [35]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [36]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split

import os

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [37]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

### Pretrain KoBERT model과 tokenizer 선언

In [38]:
#KoBERT Pretrained Tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [39]:
#KoBERT Pretrained Model
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')

In [40]:
#모델과 tokenizer import가 잘 되었는지 확인할 수 있는 코드
'''
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')
text = '[CLS] 한국어 모델을 공유합니다. [SEP]'
inputs = tokenizer.batch_encode_plus([text])
out = bertmodel(input_ids = torch.tensor(inputs['input_ids']),
            attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
'''

"\nbertmodel = BertModel.from_pretrained('skt/kobert-base-v1')\ntext = '[CLS] 한국어 모델을 공유합니다. [SEP]'\ninputs = tokenizer.batch_encode_plus([text])\nout = bertmodel(input_ids = torch.tensor(inputs['input_ids']),\n            attention_mask = torch.tensor(inputs['attention_mask']))\nout.pooler_output.shape\n"

In [43]:
#nlp module의 BERTVocab을 사용할 vocab으로 지정
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

##DATA Preprocessing

In [44]:
df1 = pd.read_csv('cafe_df_review_keward.csv')
df1
df.drop(axis=1,columns='keward',inplace= True)

In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8107 entries, 0 to 8106
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8107 non-null   object
 1   review  7799 non-null   object
 2   keward  8107 non-null   object
dtypes: object(3)
memory usage: 190.1+ KB


In [46]:
#세상에 데이터 결측치 제거
df1 = df1.dropna()

In [47]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7799 entries, 0 to 8106
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7799 non-null   object
 1   review  7799 non-null   object
 2   keward  7799 non-null   object
dtypes: object(3)
memory usage: 243.7+ KB


In [48]:
#용어에 대한 설명이 10개 이상인 용어만 사용
count = df1['id'].value_counts()
over10 = list(count[count >= 10].index)

In [49]:
#학습할 용어
over10

['중구', '남동구', '연수구', '부평구', '동구', '미추홀구', '서구']

In [50]:
df = df1[df1['id'].isin(over10)]
df

,id,review,keward
0,중구,마시안제빵소 맛있어요 뷰맛집 사람들엄청 마나서 깜\n놀 연탄빵 유명하다 줄 서있...,빵이 맛있어요
1,중구,오션뷰 카페로 워낙 유명한 베이커리카페인가봐요 !\n겨우 좋은 자리 맡아서 티타임 ...,빵이 맛있어요
2,중구,연탄빵이 유명하다는 데 도저히 대기없인 구입이 어렵네요. 포기하고 다른 빵 먹었는데...,커피가 맛있어요
3,중구,평일 오후에는 손님도 많지않고 2층에서 조용히 바다\n를 볼수있어서 너무 좋았어요...,빵이 맛있어요
4,중구,드라이브겸 나들이 장소로 좋은\n영종도 마시안제빵소-!!,빵이 맛있어요
...,...,...,...
8102,서구,굿,야외 공간이 멋져요
8103,서구,이쁜 인테리어에 숨겨진 맛있는 음료와 디저트들 음\n식 하나하나에 정성이 느껴져 먹...,커피가 맛있어요
8104,서구,분위기도 좋고 맛도 좋고 친절하셔요,빵이 맛있어요
8105,서구,좋아요~,빵이 맛있어요


In [51]:
label_list = list(df['id'].unique())
print(len(label_list))

#one-hot encoding을 위한 mapping
label_mapping = {}
for idx,term in enumerate(label_list):
  label_mapping[term] = idx

7


In [52]:
#output 도출을 위한 dict
result_dict = {}
for idx,term in enumerate(label_list):
  result_dict[idx] = term

In [53]:
df #최종적으로 사용할 데이터

,id,review,keward
0,중구,마시안제빵소 맛있어요 뷰맛집 사람들엄청 마나서 깜\n놀 연탄빵 유명하다 줄 서있...,빵이 맛있어요
1,중구,오션뷰 카페로 워낙 유명한 베이커리카페인가봐요 !\n겨우 좋은 자리 맡아서 티타임 ...,빵이 맛있어요
2,중구,연탄빵이 유명하다는 데 도저히 대기없인 구입이 어렵네요. 포기하고 다른 빵 먹었는데...,커피가 맛있어요
3,중구,평일 오후에는 손님도 많지않고 2층에서 조용히 바다\n를 볼수있어서 너무 좋았어요...,빵이 맛있어요
4,중구,드라이브겸 나들이 장소로 좋은\n영종도 마시안제빵소-!!,빵이 맛있어요
...,...,...,...
8102,서구,굿,야외 공간이 멋져요
8103,서구,이쁜 인테리어에 숨겨진 맛있는 음료와 디저트들 음\n식 하나하나에 정성이 느껴져 먹...,커피가 맛있어요
8104,서구,분위기도 좋고 맛도 좋고 친절하셔요,빵이 맛있어요
8105,서구,좋아요~,빵이 맛있어요


In [54]:
label_mapping

{'중구': 0, '연수구': 1, '동구': 2, '남동구': 3, '미추홀구': 4, '부평구': 5, '서구': 6}

##Model

###Dataset

In [55]:
train, test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 42, stratify = df['id'])

In [56]:
#Dataset 자료 형식 : [[용어1, 설명1], [용어1, 설명2], ... , [용어2, 설명1], ...] 형식의 리스트
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [57]:
#BERTDataset에 넣기 적절한 모양으로 바꿈
dataset_train = []
dataset_test = []

lab = list(train.iloc[:,0])
sen = list(train.iloc[:,1])

for l,s in zip(lab, sen):
  dataset_train.append([label_mapping[l],s])

lab2 = list(test.iloc[:,0])
sen2 = list(test.iloc[:,1])

for l,s in zip(lab2, sen2):
  dataset_test.append([label_mapping[l],s])

In [60]:
tok = tokenizer.tokenize

###Bert Model

In [61]:
#hyperparameter
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
numclasses = len(label_list)

In [62]:
data_train = BERTDataset(dataset_train, 1, 0, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 1, 0, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [63]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=numclasses,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device),
                              return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

##Training

In [64]:
model = BERTClassifier(bertmodel, dr_rate=0.1).to(device)

In [65]:
#optimization options
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [66]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [67]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [68]:
#train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train() #model을 train 모드로 설정
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-68-5b216ba6a1eb>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/195 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9470442533493042 train acc 0.125
epoch 1 train acc 0.18850289495450787


<ipython-input-68-5b216ba6a1eb>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/49 [00:00<?, ?it/s]

...

epoch 14 test acc 0.47831632653061223


  0%|          | 0/195 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.06326189637184143 train acc 0.96875
epoch 15 train acc 0.905272952853598


  0%|          | 0/49 [00:00<?, ?it/s]

epoch 15 test acc 0.4774659863945578


## Test

In [76]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

#train된 모델을 통해 output을 확인할 때 사용
def testModel(model, seq):
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, vocab, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("입력 내용의 지역은 \'"+ result_dict[idx] + '\'입니다.')
    #print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))


### Experiment

### 정확도가 47%로 높지는 않지만 실제 그 지역의 리뷰를 가지고 왔을때 그 지역을 맞추는것을 보이긴함.

실제 중구 카페리뷰

In [87]:
testModel(model, " 영종도 올 때마다 들러요. 키오스크 주문 줄이랑 소금빵 받는 줄 따로 되어있어서 사람이 많아도 체계적으로 기다릴 수 있어요. 진짜 개맛있어요 흑흑🫶🏻 너무 먹고싶어서 사진도 안찍고 먹느라 매장 사진밖에 없어요..")

입력 내용의 지역은 '중구'입니다.


실제 연수구 카페리뷰

In [93]:
testModel(model, '''튀기지 않은 도넛에 각종 토핑이 듬뿍 올라간~
송도에서 꾀 유명한~
한때는 오픈런하던 곳입니다.
원하던 도넛을 사기위해 줄서기도 했는데,
요즘은 그정도는 아닌거 같고.
이날은 주말이었는데도  운이 좋아
오후 5시 넘어서도 문이 열려있었어요.
거의 품절이고 두가지 남아있었어요.
빵이 쫄깃하고 담백하고, 그 어느 도넛보다 크고 가성비는 좋아요. 근데 맛도 아주 유니크하고 좋아요. 골라먹는 재미도 있죠~~''')

입력 내용의 지역은 '연수구'입니다.


실제 부평구 카페리뷰

In [84]:
testModel(model, '인천이라는 동네에 이렇게 이쁜 카페가 있을거라곤 생각도 못했는데🥺 너무 만족하고 나온 카페')

입력 내용의 지역은 '부평구'입니다.


실제 강화군 카페리뷰

In [89]:
testModel(model, '''매년 5월이면 데이지꽃 힐링 하러 들르는곳이에요😍 올해 데이지꽃에 일찍 폈다는 소식에 들를까말까 살짝 망설였었는데 이번주까지는 딱 예쁘게 한창이겠어요! 정말 올해도 너무 예뻐요😍👍
베이커리는 소금빵 종류가 다양해졌고 커피 메뉴도 살짝씩 달라진듯한 느낌인데 예쁘고 푸짐하네요~~
5월은 역시 마호가니 강화점이에요!
올 해도 한번 더 반하고 갑니다👍''')

입력 내용의 지역은 '동구'입니다.


실제 강화군 카페 리뷰

In [90]:
testModel(model, '''5월 22일 방문했는데 사진 찍기 딱 좋아요! 한창 피는 중이라 6월 초까지도 예쁠 것 같아요 매년 방문하는데 언제 와도 참 예쁘네요💛 뒤쪽 야외 정원으로 가도 사진 찍기 좋아요! 커피도 정말 맛있지만 작년에 아이스 전용 테이크아웃잔에 나왔는데 올해는 종이컵에 나와서 커피 양이 많이 적게 느껴져요 ㅠㅠㅠ 한입거리 느낌 ㅠㅠㅠ''')

입력 내용의 지역은 '연수구'입니다.
